In [5]:
import requests
from requests_futures.sessions import FuturesSession
import datetime as dt
import time
import sys
from drafterANN import *
import socket

api_keys = ['FE70CE9FC0D6D99279498CE852587F59','2FEC67172AAC0C393EC209A225A7E51E']
api_key_num = 1
api_key = api_keys[api_key_num]
sleep_time = 7

drafter = DotoAnn()

next_id = ''
league_id = '4266'

i = 0
errors = 0

session = FuturesSession()

def next_matches():
    url = 'https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/?key=' + api_key + '&league_id=' + league_id + '&start_at_match_id=' + str(next_id) + '&min_players=10'
    return session.get(url, timeout=4)

def next_match(match_id):
    url = 'https://api.steampowered.com/IDOTA2Match_570/GetMatchDetails/V001/?key=' + api_key + '&match_id=' + str(match_id)
    return session.get(url, timeout=4)

req = next_matches()

matches_bulk = list()

while True:
    try:
        try:
            r = req.result()
        except (requests.ConnectionError, requests.Timeout, socket.timeout):
            print("Unexpected error:", sys.exc_info()[0])
            errors += 1
            time.sleep(sleep_time)
            req = next_matches()
            continue
        
        if (r.status_code != 200):
            errors += 1
            time.sleep(sleep_time)
            req = next_matches()
            continue
        if 'matches' in r.json()['result']:
            matches = r.json()['result']['matches']
        else:
            time.sleep(sleep_time)
            req = next_matches()
            continue

        if len(matches) == 0:
            break
            
        next_id = matches[-1]['match_id'] - 1
        req = next_matches()

        for match in matches:
            m_req = next_match(match['match_id'])
            
            m = dict(id=match['match_id'])
            r_heroes = []
            d_heroes = []
            for player in match['players']:
                if player['player_slot'] < 100:
                    r_heroes.append(str(player['hero_id']))
                else:
                    d_heroes.append(str(player['hero_id']))
            m['radiant_heroes'] = ",".join(r_heroes)
            m['dire_heroes'] = ",".join(d_heroes)
            
            while True:
                try:
                    m_res = m_req.result()
                except (requests.ConnectionError, requests.Timeout, socket.timeout):
                    print("Unexpected error:", sys.exc_info()[0])
                    errors += 1
                    time.sleep(sleep_time)
                    m_req = next_match(match['match_id'])
                    continue
                if (m_res.status_code != 200):
                    errors += 1
                    time.sleep(sleep_time)
                    m_req = next_match(match['match_id'])
                    continue
                if 'radiant_win' in m_res.json()['result']:
                    rad_win = m_res.json()['result']['radiant_win']
                else:
                    time.sleep(sleep_time)
                    m_req = next_match(match['match_id'])
                    continue
                m['radiant_win'] = rad_win
                break
            matches_bulk.append(m)
    except BaseException as e:
        print("Errors: ", errors)
        raise e
            


In [3]:
from drafterANN import *
drafter = DotoAnn()

epoch_size = 3000
matches_bulk = list()

for m in Match.select().order_by(Match.seq_num.desc()).limit(epoch_size):
    match = {'radiant_heroes': m.radiant_heroes, 'dire_heroes': m.dire_heroes, 'radiant_win': m.radiant_win}
    matches_bulk.append(match)
    
print(len(matches_bulk))

3000


In [6]:
if len(matches_bulk) > 0:
    batch_xs = np.zeros((len(matches_bulk), n_input), np.int)
    batch_ys = np.zeros((len(matches_bulk), n_out), np.int)
    i2=0
    avg_certainty = 0
    for m in matches_bulk:
        xs = np.zeros((1, n_input), np.int)
        for h in m['radiant_heroes'].split(","):
            xs[0][int(h)] = 1
        for h in m['dire_heroes'].split(","):
            xs[0][int(h) + max_heroes] = 1
        certainty = abs(drafter.run(xs)[0][0] - 0.5)
        if (certainty > 0.0):
            avg_certainty += abs(drafter.run(xs)[0][0] - 0.5)

            batch_xs[i2] = xs
            batch_ys[i2][not m['radiant_win']] = 1
            i2+=1
        
    print("Accuracy", drafter.test_accuracy(batch_xs[:i2], batch_ys[:i2]))
    print("Sample", i2)
    print("Avg Certainty", avg_certainty/i2 + 0.5)

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:17: DeprecationWarning: using a boolean instead of an integer will result in an error in the future


Accuracy 0.555184
Sample 299
Avg Certainty 0.621775589809
